In [32]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from scipy.special import softmax
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import accuracy_score as ac
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVC

In [5]:
from sklearn.preprocessing import label_binarize

In [6]:
data = load_digits()
X = data.data/16
y = data.target
clases = np.unique(y)
clases

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [7]:
d = LabelBinarizer().fit_transform(y)

In [8]:
columnas = ["K","Acc.RLM"]
for g in range(2,11):
    columnas.append("Acc. SVM-Poly "+str(g))
    
tabla = pd.DataFrame(columns = columnas)

In [9]:
cv = StratifiedKFold(n_splits=10)

In [10]:
for k, (train_index,test_index) in enumerate(cv.split(X,y)):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    d_train,d_test = d[train_index],d[test_index]
    
    y_rlm_train_predict = np.zeros((train_index.shape[0], clases.shape[0]), dtype=float)
    y_rlm_test_predict = np.zeros((test_index.shape[0], clases.shape[0]), dtype=float)

    for i in range(clases.shape[0]):
        rlm = SGDRegressor()
        rlm.fit(X_train, d_train[:,i])
        y_rlm_train_predict[:,i] = rlm.predict(X_train)
        y_rlm_test_predict[:,i] = rlm.predict(X_test)
    X_svm_train = softmax(y_rlm_train_predict, axis=1)
    X_svm_test = softmax(y_rlm_test_predict, axis=1)
    acc_rlm = ac(np.argmax(X_svm_test, axis=1),y_test)
    
    acc_svm = np.zeros(9,dtype=float)
    
    for f,grd in enumerate(range(2,11)):
        svm = SVC(kernel="poly", degree = grd)
        svm.fit(X_svm_train,y_train)
        acc_svm[f] = svm.score(X_svm_test, y_test)
    tabla.loc[k] = [k,acc_rlm]+[acc_svm[f] for f in range(9)]
tabla

,K,Acc.RLM,Acc. SVM-Poly 2,Acc. SVM-Poly 3,Acc. SVM-Poly 4,Acc. SVM-Poly 5,Acc. SVM-Poly 6,Acc. SVM-Poly 7,Acc. SVM-Poly 8,Acc. SVM-Poly 9,Acc. SVM-Poly 10
0,0.0,0.938889,0.944444,0.938889,0.938889,0.933333,0.933333,0.933333,0.927778,0.927778,0.933333
1,1.0,0.955556,0.972222,0.961111,0.961111,0.961111,0.961111,0.966667,0.966667,0.972222,0.972222
2,2.0,0.888889,0.927778,0.916667,0.916667,0.916667,0.922222,0.922222,0.916667,0.922222,0.922222
3,3.0,0.861111,0.911111,0.916667,0.916667,0.911111,0.911111,0.900000,0.894444,0.894444,0.900000
4,4.0,0.900000,0.933333,0.955556,0.955556,0.955556,0.955556,0.955556,0.955556,0.955556,0.955556
5,5.0,0.888889,0.972222,0.972222,0.972222,0.972222,0.961111,0.961111,0.961111,0.966667,0.966667
6,6.0,0.955556,0.972222,0.983333,0.983333,0.983333,0.983333,0.977778,0.977778,0.977778,0.977778
7,7.0,0.949721,0.960894,0.960894,0.960894,0.960894,0.972067,0.966480,0.966480,0.966480,0.966480
8,8.0,0.843575,0.882682,0.893855,0.888268,0.893855,0.899441,0.899441,0.899441,0.899441,0.893855
9,9.0,0.910615,0.960894,0.932961,0.938547,0.938547,0.938547,0.944134,0.944134,0.949721,0.949721


In [28]:
i = 2
j = 0
xd = [0,0,0,0,0,0,0,0,0,0,0]
for i in range(11):
    if(i>=2):
        media = tabla.mean().values[i]
        xd[j] = media
        j = j+1
xd

[0.943780260707635,
 0.9432153941651148,
 0.9432153941651148,
 0.9426629422718806,
 0.9437833643699565,
 0.9426722532588453,
 0.9410055865921787,
 0.9432309124767224,
 0.9437833643699565,
 0,
 0]

In [31]:
print("El grado del polinomio de la svm optimo", xd.index(max(xd)))

El grado del polinomio de la svm optimo 4
